In [ ]:
import schedule
import time
from datetime import datetime, timedelta
from confluent_kafka import Producer
import pandas as pd

p = Producer({'bootstrap.servers': 'localhost:9092'})

topics = ["PFM-1"]


def wikipediaRequest():
    frames = []
    for topic in topics:
        print(topic)
        start = (datetime.now() - timedelta(1)).strftime('%Y%m%d')
        end = (datetime.now() - timedelta(1)).strftime('%Y%m%d')
        wiki_url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/{topic}/daily/{start}/{end}'
        frames.append(pd.read_json(wiki_url)) 
    return pd.concat(frames)


def getNewWikipediaData():
    df = wikipediaRequest()
    print('Sending new data', df.to_json(orient='records'))
    p.produce('wiki', key='myKey', value=bytes(df.to_json(orient='records'), 'utf-8'))
    

current_time = datetime.now()
future_time = current_time + timedelta(minutes=1)
print(future_time.strftime("%H:%M"))
schedule.every().day.at(future_time.strftime("%H:%M")).do(getNewWikipediaData)

while True:
    schedule.run_pending()
    time.sleep(1)

23:43
PFM-1
Sending new data [{"items":{"project":"en.wikipedia","article":"PFM-1","granularity":"daily","timestamp":"2022032000","access":"desktop","agent":"user","views":205}}]
